In [10]:
#Example
using Pkg
# Pkg.add("Flux")
# Pkg.add("MLDatasets")
# Pkg.add("CUDA")
Pkg.add("cuDNN")

   Resolving package versions...


   Installed CUDNN_jll ─ v8.9.2+0
   Installed cuDNN ───── v1.1.0


    Updating `C:\Users\maria\.julia\environments\v1.9\Project.toml`
  [02a925ec] + cuDNN v1.1.0
    Updating `C:\Users\maria\.julia\environments\v1.9\Manifest.toml`


  [02a925ec] + cuDNN v1.1.0
  [62b44479] + CUDNN_jll v8.9.2+0


Precompiling

project...

  ✓ CUDNN_jll


  ✓ cuDNN


  ✓ NNlib → NNlibCUDACUDNNExt


  ✓ Flux → FluxCUDAcuDNNExt
  4 dependencies successfully precompiled in 35 seconds. 275 already precompiled.


In [11]:
using Flux, MLDatasets, CUDA, FileIO
using Flux: train!, onehotbatch

In [15]:
# Load training data (images, labels)
using Flux, MLDatasets, CUDA
using Flux: train!, onehotbatch

x_train, y_train = MLDatasets.MNIST.traindata()
# Load test data (images, labels)
x_test, y_test = MLDatasets.MNIST.testdata()
# Convert grayscale to float
x_train = Float32.(x_train)
# Create labels batch
y_train = Flux.onehotbatch(y_train, 0:9)

┌ Warning: MNIST.traindata() is deprecated, use `MNIST(split=:train)[:]` instead.
└ @ MLDatasets C:\Users\maria\.julia\packages\MLDatasets\Ob4aN\src\datasets\vision\mnist.jl:187


┌ Warning: MNIST.testdata() is deprecated, use `MNIST(split=:test)[:]` instead.
└ @ MLDatasets C:\Users\maria\.julia\packages\MLDatasets\Ob4aN\src\datasets\vision\mnist.jl:195


10×60000 OneHotMatrix(::Vector{UInt32}) with eltype Bool:
 ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  …  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  1  ⋅  ⋅  1  ⋅  1  ⋅  ⋅  ⋅  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅     ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  1  ⋅  1     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅
 ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  …  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅  1  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅     1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅     ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅  1
 ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅     ⋅  ⋅  1  ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅

In [16]:
model = Chain(
    Dense(784, 128, relu),
    Dense(128, 64, relu),
    Dense(64, 10, relu), softmax
)

Chain(
  Dense(784 => 128, relu),              # 100_480 parameters
  Dense(128 => 64, relu),               # 8_256 parameters
  Dense(64 => 10, relu),                # 650 parameters
  NNlib.softmax,
)                   # Total: 6 arrays, 109_386 parameters, 427.664 KiB.

In [17]:
loss(x, y) = Flux.Losses.logitcrossentropy(model(x), y)

loss (generic function with 1 method)

In [26]:
using Flux

optimizer = ADAM(0.0001)
parameters = Flux.params(model)
# flatten() function converts array 28x28x60000 into 784x60000 (28*28x60000)
train_data = [(Flux.flatten(x_train), Flux.flatten(y_train))]
# Range in loop can be used smaller
for i in 1:400
    Flux.train!(loss, parameters, train_data, optimizer)
end

In [27]:
test_data = [(Flux.flatten(x_test), y_test)]
accuracy = 0
for i in 1:length(y_test)
    if findmax(model(test_data[1][1][:, i]))[2] - 1  == y_test[i]
        accuracy = accuracy + 1
    end
end
println(accuracy / length(y_test))

0.7674
